# Tensorflow로 CNN 모델 구성하기

### 활성화 함수 : adam
### 배치정규화, 드롭아웃( 0.3 ), earlystopping 적용

In [1]:
# 1. 필요한 패키지 가져오는 코드 

import tensorflow as tf   # 텐써 플로우 2.0 
from tensorflow.keras.datasets.mnist import load_data  # 텐써플로우에 내장되어있는 
                                                                         # mnist 데이터를 가져온다.
from tensorflow.keras.models import Sequential  # 모델을 구성하기 위한 모듈
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout  # 완전 연결계층을 구성하기 위한 모듈
from tensorflow.keras.utils import to_categorical # one encoding 하는 모듈
import  numpy as  np

tf.random.set_seed(777)

(x_train, y_train), (x_test, y_test) = load_data(path='mnist.npz')  # mnist 데이터 로드
print(x_train.shape) # (60000, 28, 28)  
print(x_test.shape) # (10000, 28, 28)    



11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(10000, 28, 28)


In [2]:

#2. 색조까지 포함한 4차원 shape 로 reshape 해야합니다.

# 3차원(60000x28x28) ----> 4차원( ? x 28 x 28 x 1 ) 을 변경하는데 안의 요소(픽셀)의 갯수는 동일해야합니다.
x_train = x_train.reshape(-1, 28, 28, 1)
print(x_train.shape)  # (60000, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)
print(x_test.shape) # (10000, 28, 28, 1)

# 2. 정규화 진행  
x_train = x_train / 255   # 0~1 사이의 데이터로 변경합니다. 
x_test = x_test / 255

# 3. 정답 데이터를 준비한다. 
# 하나의 숫자를 one hot encoding 한다. (예:  4 ---> 0 0 0 0 1 0 0 0 0 0 ) 
y_train = to_categorical(y_train)  # 훈련 데이터의 라벨(정답)을 원핫 인코딩
y_test = to_categorical(y_test)    # 테스트 데이터의 라벨(정답)을 원핫 인코딩 


(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [3]:

# 4. 모델을 구성합니다. 3층 신경망으로 구성
model = Sequential()
model.add(Conv2D(100, kernel_size=(5,5), input_shape=(28,28,1),  activation='relu', padding = 'same')  ) 
model.add( BatchNormalization() )
model.add(MaxPooling2D(pool_size=(2,2), padding='same') ) # 이미지를 선명하게 해주는 층
model.add( Flatten() )  # 완전연결계층에 들어갈 수 있도록 이미지(피쳐맵)를 1차원으로 변경 
model.add(Dense(50, activation = 'sigmoid', input_shape = (784, )))  # 1층 완전연결계층
model.add( BatchNormalization() )
model.add( Dropout(0.3) )
model.add(Dense(50, activation = 'sigmoid') ) # 2층 은닉층 
model.add( BatchNormalization() )
model.add( Dropout(0.3) )
model.add(Dense(10, activation = 'softmax'))  # 3층 출력층 

# 5. 모델을 설정합니다. ( 경사하강법, 오차함수를 정의해줍니다. )
model.compile(optimizer='adam', 
                     loss = 'categorical_crossentropy', 
                     metrics=['acc'])  # 학습과정에서 정확도를 보려고 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 100)       2600      
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 100)      400       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 100)      0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 19600)             0         
                                                                 
 dense (Dense)               (None, 50)                980050    
                                                                 
 batch_normalization_1 (Batc  (None, 50)               2

In [4]:

#6. 모델을 훈련시킵니다. 

from tensorflow.keras.callbacks import EarlyStopping

callbacks = [ EarlyStopping( monitor = 'val_acc', patience = 5 , verbose = 1 ) ]

history = model.fit(x_train, y_train, 
                         epochs = 30,  # 30에폭
                         batch_size = 100,
                         validation_data=(x_test, y_test),
                         callbacks = callbacks )



Epoch 1/30
600/600 [==============================] - 146s 238ms/step - loss: 0.2094 - acc: 0.9412 - val_loss: 0.0700 - val_acc: 0.9774
Epoch 2/30
600/600 [==============================] - 139s 231ms/step - loss: 0.0786 - acc: 0.9781 - val_loss: 0.0554 - val_acc: 0.9822
Epoch 3/30
600/600 [==============================] - 140s 234ms/step - loss: 0.0585 - acc: 0.9828 - val_loss: 0.0520 - val_acc: 0.9853
Epoch 4/30
600/600 [==============================] - 140s 234ms/step - loss: 0.0488 - acc: 0.9853 - val_loss: 0.0558 - val_acc: 0.9840
Epoch 5/30
600/600 [==============================] - 139s 232ms/step - loss: 0.0389 - acc: 0.9884 - val_loss: 0.0460 - val_acc: 0.9868
Epoch 6/30
600/600 [==============================] - 140s 233ms/step - loss: 0.0364 - acc: 0.9891 - val_loss: 0.0366 - val_acc: 0.9883
Epoch 7/30
600/600 [==============================] - 139s 232ms/step - loss: 0.0314 - acc: 0.9907 - val_loss: 0.0407 - val_acc: 0.9890
Epoch 8/30
600/600 [============================

In [ ]:
# 7.모델을 평가합니다. (오차, 정확도가 출력됩니다.)
model.evaluate(x_test, y_test)

train_acc_list=history.history['acc']
train_acc_list

test_acc_list=history.history['val_acc']
test_acc_list

import  matplotlib.pyplot  as  plt

x = np.arange( len(train_acc_list) )
plt.plot( x, train_acc_list, label='train acc')
plt.plot( x, test_acc_list, label='test acc',  linestyle='--')
plt.ylim(0, 1)
plt.legend(loc='lower right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()